In [1]:
import pandas as pd
import os
from dotenv import load_dotenv

In [2]:
def load_config():
    """Loads configuration from environment variables."""
    load_dotenv()
    return {
        "PROJECT_ID": os.getenv("PROJECT_ID"),
        "LOCATION": os.getenv("LOCATION"),
        "INPUT_BUCKET": os.getenv("INPUT_BUCKET"),
        "OUTPUT_BUCKET": os.getenv("OUTPUT_BUCKET"),
        "PROCESSOR_ID": os.getenv("PROCESSOR_ID"),
    }

In [3]:
config = load_config()
print(config)

{'PROJECT_ID': 'credemhack-cloudfunctions', 'LOCATION': 'us', 'INPUT_BUCKET': 'credemhack_cloud_fuctions', 'OUTPUT_BUCKET': 'your-export-bucket-name', 'PROCESSOR_ID': 'e4a86664fd2377e2'}


### ETL dopo l'estrazione OCR

In [4]:
df = pd.read_csv("../data/extracted/extracted_data.csv")
df_personale = pd.read_csv("../data/Elenco Personale.xlsx - Foglio 1.csv")

In [5]:
df_results = df.copy(deep=True)

In [6]:
df_personale.head()

,Cognome,Nome,Person Number
0,AGNELLI,DAVIDE,999930919
1,AGOSTINO,FEDERICA,999931077
2,ALBERGHINI,CARLOTTA,999930708
3,ALBERGHINI,ELISA,999930789
4,ALBERGHINI,MATTEO,999930781


In [7]:
df_results.head()

,File_Name,Nome,Cognome,Data,Cluster,Country
0,0009554846001_2169778.pdf,ERRORE,Protti,2018-09-14,Part-time,Italy
1,0004837674001_2010435.TIF,ARIANNA,ERRORE,2014-06-19,Trasferimento,Italy
2,0004856283001_2005578.TIF,Elisabetta,Diegoli,ERRORE,Part-time,Italy
3,0004743991001_55840.TIF,SILVIA,VECCHI,2014-03-01,Assegnazione ruolo,ERRORE
4,0009584812001_2169778.PDF,Silvia,Antonioli,2018-10-23,Proroga TD,Italy


In [8]:
import pandas as pd
import numpy as np

def clean_registry_df(
    df: pd.DataFrame,
    *,
    name_cols=("Nome", "Cognome"),   # columns to upper-case
    date_col="Data",
    country_col="Country"            # change if your column is called differently
) -> pd.DataFrame:
    """
    • Replace every spelling/spacing/casing of 'ERRORE' (and NaN/None) with placeholders  
    • Convert valid dates → YYYY/MM/DD, invalid → 'ERRORE'  
    • Upper-case names + country (placeholders already all-caps)  
    Returns a *new* DataFrame.
    """

    # 1️⃣  normalise any variant of "ERRORE"
    df = df.applymap(
        lambda x: "ERRORE"
        if isinstance(x, str) and x.strip().upper() == "ERRORE"
        else x
    )

    # 2️⃣  placeholders for key columns
    placeholders = {
        "Nome": "NONAME",
        "Cognome": "NOLASTNAME",
        "Data": "NODATE",
        country_col: "",
    }
    for col, ph in placeholders.items():
        if col in df.columns:
            df[col] = (
                df[col]
                .fillna("ERRORE")
                .replace("ERRORE", ph)
            )

    # 3️⃣  robust date normalisation → YYYY/MM/DD
    if date_col in df.columns:
        def _format_date(v):
            if v in ("NODATE", "ERRORE"):
                return v
            try:
                dt = pd.to_datetime(v, errors="raise", dayfirst=False, utc=False)
                return dt.strftime("%Y/%m/%d")
            except Exception:
                return "ERRORE"

        df[date_col] = df[date_col].apply(_format_date)

    # 4️⃣  UPPER-case names
    for col in name_cols:
        if col in df.columns:
            ph = placeholders.get(col)
            df[col] = df[col].apply(
                lambda s: s if s == ph else str(s).strip().upper()
            )

    # 5️⃣   Capitalize country with capitalize
    if country_col in df.columns:
        df[country_col] = df[country_col].apply(
            lambda s: s if s == ph else str(s).strip().capitalize()
        )
    
    return df


df_results = clean_registry_df(df_results)
df_results.head()

C:\Users\PC-EC\AppData\Local\Temp\ipykernel_26440\2842430550.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(


,File_Name,Nome,Cognome,Data,Cluster,Country
0,0009554846001_2169778.pdf,NONAME,PROTTI,2018/09/14,Part-time,Italy
1,0004837674001_2010435.TIF,ARIANNA,NOLASTNAME,2014/06/19,Trasferimento,Italy
2,0004856283001_2005578.TIF,ELISABETTA,DIEGOLI,NODATE,Part-time,Italy
3,0004743991001_55840.TIF,SILVIA,VECCHI,2014/03/01,Assegnazione ruolo,
4,0009584812001_2169778.PDF,SILVIA,ANTONIOLI,2018/10/23,Proroga TD,Italy


In [9]:
# creiamo il daframe finale, con le colonne: FILENAME  | METADATA|DocumentsOfRecord|PersonNumber|DocumentType|Country|DocumentCode|DocumentName|DateFrom|DateTo|SourceSystemOwner|SourceSystemId




In [10]:
# creiamo il df della sezione 1
cols_section_1 = [
    "FILENAME",
    "METADATA",
    "DocumentsOfRecord",
    "PersonNumber",
    "DocumentType",
    "Country",
    "DocumentCode",
    "DocumentName",
    "DateFrom",
    "DateTo",
    "SourceSystemOwner",
    "SourceSystemId"
]
df_section_1 = pd.DataFrame(columns=cols_section_1)
df_section_1.head()

,FILENAME,METADATA,DocumentsOfRecord,PersonNumber,DocumentType,Country,DocumentCode,DocumentName,DateFrom,DateTo,SourceSystemOwner,SourceSystemId


## Campi da riempire:
- FILENAME: nome del file
- METADATA: MERGE (valore unico per tutti)
- DocumentsOfRecords: DocumentsOfRecords (valore fisso)
- PersonNumber: da prendere nel df_personale
- DocumentType: da Cluster (df_results) - check categorie da df_personale
- Country: da Country in df_results: vuoto se Country è ERRORE
- DocumentCode: ID univoco composto da: PersonNumber_DateFrom_DocumentType. DateFrom in formato YYYYMMDD.
- DocumentName: NOME COGNOME da df_results nome e cognome, tutto maiuscolo, spazio singolo di intermezzo tra nome e cognome. Nome e Cognome vanno trovati, se non matchano, c'è la condizione di "riga speciale" con valori predefiniti
- DateFrom: da Data in df_results, formato da trasformare in YYYY/MM/DD oppure NODATE se è ERRORE
- DateTo: nullo (lasciare vuoto)
- SourceSystemOwner: PEOPLE (unico valore su tutte le colonne)
- SourceSystemId: uguale al DocumentCode

## Riga speciale:
Se non ho il match, ho i seguenti campi riempiti:
- PersonNumber: Nessun dipendente
- DocumentType: SCARTATO
- Country: campo vuoto(niente da mettere)
- DocumentCode: ID univoco composto da: PersonNumber_DateFrom_DocumentType. DateFrom in formato YYYYMMDD.
- DocumentName: Nessun dipendente
- DateFrom: da Data in df_results, formato da trasformare in YYYY/MM/DD oppure NODATE se è ERRORE
- DateTo: nullo (lasciare vuoto)
- SourceSystemOwner: PEOPLE (unico valore su tutte le colonne)
- SourceSystemId: uguale al DocumentCode

In [17]:
# iteriamo su df_results
def combine_clean_data(df_results, df_personale):

    cols_section_1 = [
    "FILENAME",
    "METADATA",
    "DocumentsOfRecord",
    "PersonNumber",
    "DocumentType",
    "Country",
    "DocumentCode",
    "DocumentName",
    "DateFrom",
    "DateTo",
    "SourceSystemOwner",
    "SourceSystemId"
    ]
    df_section_1 = pd.DataFrame(columns=cols_section_1)

    cols_section_2 = cols_section_1 + ["DataTypeCode", "URLorTextorFileName", "Title", "File"]
    df_section_2 = pd.DataFrame(columns=cols_section_2)

        
    for _, row_results in df_results.iterrows():
        # 1. Cerchiamo il match e dividiamo in due casi: match o non match
        # Per cercare il match, cerchiamo il match tra Nome e Cognome in df_personale. Se non c'è, allora è una riga speciale.
        # Riga speciale costruita con valori specifici e altri no.
        # 2. Se match, aggiungiamo i dati in df_section_1 i dati.
        match = False
        for _, row_personale in df_personale.iterrows():
            nome_pers = row_personale.get("Nome", "NONAME").strip().upper()
            cognome_pers = row_personale.get("Cognome", "NOLASTNAME").strip().upper()

            nome_res = row_results.get("Nome", "NONAME").strip().upper()
            cognome_res = row_results.get("Cognome", "NOLASTNAME").strip().upper()
            data_res = row_results.get("Data", "NODATE")

            if nome_res == nome_pers and cognome_res == cognome_pers and data_res != "NODATE":
                # match
                match = True
                person_number = row_personale["Person Number"]
                document_type = row_results.get("Cluster", "Nessun cluster")
                country = row_results.get("Country", "")
                document_name = f"{nome_res} {cognome_res}".strip().upper()

        
        if not match:
            # non match
            person_number = "Nessun dipendente"
            document_type = "SCARTATO"
            country = ""
            document_name = "Nessun dipendente"
            document_code = "Nessun dipendente"

        # aggiungiamo i dati in comune per match e non match
        file_name = row_results["File_Name"]
        metadata = "MERGE"
        documents_of_records = "DocumentsOfRecords"
        date_from = row_results["Data"]
        date_normalized = date_from.replace("/", "").strip()
        document_code = f"{person_number}_{date_normalized}_{document_type}"
        date_to = ""
        source_system_owner = "PEOPLE"
        source_system_id = document_code

        # aggiungiamo i dati per la sezione 1
        # non usare append
        df_section_1.loc[len(df_section_1)] = [file_name, metadata, documents_of_records, person_number, document_type, country, document_code, document_name, date_from, date_to, source_system_owner, source_system_id]

        #salviamo la riga per il df_section_2

        data_type_code = "FILE"
        url_or_text_or_file_name = file_name
        title = file_name
        file = file_name

        df_section_2.loc[len(df_section_2)] = [file_name, metadata, documents_of_records, person_number, document_type, country, document_code, document_name, date_from, date_to, source_system_owner, source_system_id, data_type_code, url_or_text_or_file_name, title, file]

    return df_section_1, df_section_2

In [18]:
df_sec_1, df_sec_2 = combine_clean_data(df_results, df_personale)
df_sec_1.head()

,FILENAME,METADATA,DocumentsOfRecord,PersonNumber,DocumentType,Country,DocumentCode,DocumentName,DateFrom,DateTo,SourceSystemOwner,SourceSystemId
0,0009554846001_2169778.pdf,MERGE,DocumentsOfRecords,Nessun dipendente,SCARTATO,,Nessun dipendente_20180914_SCARTATO,Nessun dipendente,2018/09/14,,PEOPLE,Nessun dipendente_20180914_SCARTATO
1,0004837674001_2010435.TIF,MERGE,DocumentsOfRecords,Nessun dipendente,SCARTATO,,Nessun dipendente_20140619_SCARTATO,Nessun dipendente,2014/06/19,,PEOPLE,Nessun dipendente_20140619_SCARTATO
2,0004856283001_2005578.TIF,MERGE,DocumentsOfRecords,Nessun dipendente,SCARTATO,,Nessun dipendente_NODATE_SCARTATO,Nessun dipendente,NODATE,,PEOPLE,Nessun dipendente_NODATE_SCARTATO
3,0004743991001_55840.TIF,MERGE,DocumentsOfRecords,999930948,Assegnazione ruolo,,999930948_20140301_Assegnazione ruolo,SILVIA VECCHI,2014/03/01,,PEOPLE,999930948_20140301_Assegnazione ruolo
4,0009584812001_2169778.PDF,MERGE,DocumentsOfRecords,999931008,Proroga TD,Italy,999931008_20181023_Proroga TD,SILVIA ANTONIOLI,2018/10/23,,PEOPLE,999931008_20181023_Proroga TD


In [19]:
df_sec_2.head()

,FILENAME,METADATA,DocumentsOfRecord,PersonNumber,DocumentType,Country,DocumentCode,DocumentName,DateFrom,DateTo,SourceSystemOwner,SourceSystemId,DataTypeCode,URLorTextorFileName,Title,File
0,0009554846001_2169778.pdf,MERGE,DocumentsOfRecords,Nessun dipendente,SCARTATO,,Nessun dipendente_20180914_SCARTATO,Nessun dipendente,2018/09/14,,PEOPLE,Nessun dipendente_20180914_SCARTATO,FILE,0009554846001_2169778.pdf,0009554846001_2169778.pdf,0009554846001_2169778.pdf
1,0004837674001_2010435.TIF,MERGE,DocumentsOfRecords,Nessun dipendente,SCARTATO,,Nessun dipendente_20140619_SCARTATO,Nessun dipendente,2014/06/19,,PEOPLE,Nessun dipendente_20140619_SCARTATO,FILE,0004837674001_2010435.TIF,0004837674001_2010435.TIF,0004837674001_2010435.TIF
2,0004856283001_2005578.TIF,MERGE,DocumentsOfRecords,Nessun dipendente,SCARTATO,,Nessun dipendente_NODATE_SCARTATO,Nessun dipendente,NODATE,,PEOPLE,Nessun dipendente_NODATE_SCARTATO,FILE,0004856283001_2005578.TIF,0004856283001_2005578.TIF,0004856283001_2005578.TIF
3,0004743991001_55840.TIF,MERGE,DocumentsOfRecords,999930948,Assegnazione ruolo,,999930948_20140301_Assegnazione ruolo,SILVIA VECCHI,2014/03/01,,PEOPLE,999930948_20140301_Assegnazione ruolo,FILE,0004743991001_55840.TIF,0004743991001_55840.TIF,0004743991001_55840.TIF
4,0009584812001_2169778.PDF,MERGE,DocumentsOfRecords,999931008,Proroga TD,Italy,999931008_20181023_Proroga TD,SILVIA ANTONIOLI,2018/10/23,,PEOPLE,999931008_20181023_Proroga TD,FILE,0009584812001_2169778.PDF,0009584812001_2169778.PDF,0009584812001_2169778.PDF
